In [ ]:
import numpy as np
from copy import deepcopy
from random import random, randint
from tqdm.auto import tqdm

def simulated_annealing_knapsack(
    VALUES, WEIGHTS, CONSTRAINTS,
    max_steps=10000, temp_init=1000, temp_decay=0.99, tweak_strength=0.6
):
    NUM_ITEMS = len(VALUES)
    NUM_KNAPSACKS = len(CONSTRAINTS)
    NUM_DIMENSIONS = WEIGHTS.shape[1]

    # Soluzione iniziale basata su valore/peso
    def generate_valid_initial_solution():
        solution = -np.ones(NUM_ITEMS, dtype=int)
        usage = np.zeros_like(CONSTRAINTS)
        value_density = VALUES / (np.sum(WEIGHTS, axis=1) + 1e-6)
        item_order = np.argsort(-value_density)

        for i in item_order:
            knapsacks = list(range(NUM_KNAPSACKS))
            np.random.shuffle(knapsacks)
            for k in knapsacks:
                if np.all(usage[k] + WEIGHTS[i] <= CONSTRAINTS[k]):
                    solution[i] = k
                    usage[k] += WEIGHTS[i]
                    break
        return solution

    def is_valid(solution):
        usage = np.zeros_like(CONSTRAINTS)
        for i in range(NUM_ITEMS):
            k = solution[i]
            if k >= 0:
                usage[k] += WEIGHTS[i]
        return np.all(usage <= CONSTRAINTS)

    def total_value(solution):
        return np.sum([VALUES[i] for i in range(NUM_ITEMS) if solution[i] >= 0])

    # Penalità proporzionale per violazioni
    def cost(solution):
        usage = np.zeros_like(CONSTRAINTS)
        for i in range(NUM_ITEMS):
            k = solution[i]
            if k >= 0:
                usage[k] += WEIGHTS[i]
        overuse = np.maximum(usage - CONSTRAINTS, 0)
        penalty = np.sum(overuse)
        return total_value(solution) - penalty * 10

    # Tweak intelligente: sposta solo se non viola
    def tweak(solution):
        new_solution = deepcopy(solution)
        usage = np.zeros_like(CONSTRAINTS)
        for i in range(NUM_ITEMS):
            k = solution[i]
            if k >= 0:
                usage[k] += WEIGHTS[i]

        for _ in range(int(tweak_strength * NUM_ITEMS)):
            i = randint(0, NUM_ITEMS - 1)
            original_k = new_solution[i]
            candidate_knapsacks = list(range(NUM_KNAPSACKS))
            if 0 <= original_k < NUM_KNAPSACKS:
                candidate_knapsacks.remove(original_k)
            np.random.shuffle(candidate_knapsacks)
            for new_k in candidate_knapsacks:
                if np.all(usage[new_k] + WEIGHTS[i] <= CONSTRAINTS[new_k]):
                    usage[original_k] -= WEIGHTS[i]
                    usage[new_k] += WEIGHTS[i]
                    new_solution[i] = new_k
                    break
        return new_solution

    current_solution = generate_valid_initial_solution()
    current_cost = cost(current_solution)
    temp = temp_init
    best_solution = deepcopy(current_solution)
    best_cost = current_cost

    for step in tqdm(range(max_steps)):
        if step % 100 == 0:
            temp *= temp_decay

        new_solution = tweak(current_solution)
        new_cost = cost(new_solution)

        if new_cost > current_cost or random() < np.exp((new_cost - current_cost) / temp):
            current_solution = new_solution
            current_cost = new_cost
            if new_cost > best_cost:
                best_solution = deepcopy(new_solution)
                best_cost = new_cost

    return best_solution, best_cost


In [ ]:
# TC1
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

sol1, val1 = simulated_annealing_knapsack(VALUES, WEIGHTS, CONSTRAINTS)
print("TC1 Value:", val1)

# TC2
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(2000, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

sol2, val2 = simulated_annealing_knapsack(VALUES, WEIGHTS, CONSTRAINTS, max_steps=2000, temp_init=200, tweak_strength=0.2)
print("TC2 Value:", val2)

# TC3
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(10000, 2000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

sol3, val3 = simulated_annealing_knapsack(VALUES, WEIGHTS, CONSTRAINTS, max_steps=3000, temp_init=500, tweak_strength=0.1)
print("TC3 Value:", val3)